In [4]:
import pandapower as pp
import pandapower.networks
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import time
%matplotlib inline

In [5]:
def rename(network):
    line_name = []
    bus_name = []
    for i in (network.line.index.values+1):
        line_name.append('line_'+str(i))
    
    for j in network.bus.name.values:
        bus_name.append('bus_'+str(j))
    
    # Rename lines: i.e. 'line_1'
    network.line.name=line_name
    
    # Rename bus: i.e. 'bus_1'
    network.bus.name=bus_name
    pass

def record_pmu_data(network, pmu_index):
    
    pmu_0_va_degree.append(network.res_bus.loc[pmu_index[0]].va_degree)
    pmu_0_vm_pu.append(network.res_bus.loc[pmu_index[0]].vm_pu)
    
    pmu_1_va_degree.append(network.res_bus.loc[pmu_index[1]].va_degree)
    pmu_1_vm_pu.append(network.res_bus.loc[pmu_index[1]].vm_pu)
    
    pmu_2_va_degree.append(network.res_bus.loc[pmu_index[2]].va_degree)
    pmu_2_vm_pu.append(network.res_bus.loc[pmu_index[2]].vm_pu)

def change_loads(network, loads_at_time_t):
    for i in range(network.load.shape[0]):
        network.load.p_kw[i] = loads_at_time_t[i]

In [96]:
pmu_location = [23, 26, 29]
pmu_index = [i-1 for i in pmu_location] # To conform with data format in the case, 0-start

In [171]:
mean = 300000
std = 10000
samples_per_second = 60
#lines_to_be_outed = [4, 14, 34]
lines_to_be_outed = [1, 4, 7, 10, 14, 18, 21, 24, 30, 34]
samples_per_line = 100

In [172]:
training_data = pd.DataFrame()
start_time = time.time()

for line in lines_to_be_outed:
    for line_sample in range(samples_per_line):
        
        # Set up data structure for recording
        pmu_0_va_degree = []
        pmu_1_va_degree = []
        pmu_2_va_degree = []

        pmu_0_vm_pu = []
        pmu_1_vm_pu = []
        pmu_2_vm_pu = []

        pmu_data = pd.DataFrame()
        for pmu_loc in range(len(pmu_location)):
            pmu_data['pmu_'+str(pmu_loc)+'_va_degree'] = []
            pmu_data['pmu_'+str(pmu_loc)+'_vm_pu'] = []

            
        # Set up power network
        net_39 = pandapower.networks.case39()
        rename(net_39)
        
        # Generate a new set of loads for one outage instance
        loads = pd.DataFrame()
        for load_index in range(net_39.load.shape[0]):
            name = 'load_'+str(load_index)
            loads[name] = np.random.normal(loc=mean, scale=std, size=samples_per_second)
        
        # Run the power network
        pp.runpp(net_39)

        # Record data, for a PMU with 60 sps
        for timestep in range(samples_per_second):
            change_loads(net_39, loads.loc[timestep])
            pp.runpp(net_39)
            record_pmu_data(net_39, pmu_index)

        pmu_data = pd.DataFrame(list(zip(pmu_0_va_degree, pmu_0_vm_pu, 
                                    pmu_1_va_degree, pmu_1_vm_pu, pmu_2_va_degree, 
                                    pmu_2_vm_pu)), columns=pmu_data.columns)

        # Setting a line out of service
        line_number = line
        net_39.line.in_service.loc[line_number-1] = False
        net_39.line.loc[line_number-1]

        # Record data after the line outage
        for timestep in range(samples_per_second):
            change_loads(net_39, loads.loc[timestep])
            pp.runpp(net_39)
            record_pmu_data(net_39, pmu_index)

        pmu_data = pd.DataFrame(list(zip(pmu_0_va_degree, pmu_0_vm_pu, 
                                    pmu_1_va_degree, pmu_1_vm_pu, pmu_2_va_degree, 
                                    pmu_2_vm_pu)), columns=pmu_data.columns)


        fft_pmu0 = np.fft.fft(pmu_data.pmu_0_va_degree)
        fft_pmu1 = np.fft.fft(np.asarray(pmu_data.pmu_1_va_degree)-np.asarray(pmu_data.pmu_0_va_degree))
        fft_pmu2 = np.fft.fft(np.asarray(pmu_data.pmu_2_va_degree)-np.asarray(pmu_data.pmu_0_va_degree))


        imag_pmu0 = fft_pmu0.imag.tolist()
        imag_pmu1 = fft_pmu1.imag.tolist()
        imag_pmu2 = fft_pmu2.imag.tolist()

        predictor = imag_pmu0[60:]
        predictor.append(imag_pmu0[60] - imag_pmu0[59])
        predictor.extend(imag_pmu1[60:])
        predictor.append(imag_pmu1[60] - imag_pmu1[59])
        predictor.extend(imag_pmu2[60:])
        predictor.append(imag_pmu2[60] - imag_pmu2[59])

        training = predictor
        training.extend([line_number])
        training_data['line_'+str(line)+'_'+str(line_sample)] = pd.Series(training)

duration = time.time() - start_time

print('Outed lines: ', lines_to_be_outed)
print('Number of outed lines simulated: %.f' %len(lines_to_be_outed))
print('Number of simulation per line: %.f' %samples_per_line)
print('PMU records %.f samples per second' %samples_per_second)
print('Time taken: %.3fs' %duration)

Outed lines:  [1, 4, 7, 10, 14, 18, 21, 24, 30, 34]
Number of outed lines simulated: 10
Number of simulation per line: 100
PMU records 60 samples per second
Time taken: 2805.402s


## Train classifier

In [173]:
train_df = training_data.transpose()

train_df.reset_index(drop=True, inplace=True)

# load data
X = train_df.iloc[:,:-1]
y = train_df.iloc[:,-1:]

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import f1_score
from sklearn import linear_model

logreg = linear_model.LogisticRegression(C=1e5, penalty='l1', solver='saga', multi_class='multinomial')

sss = StratifiedShuffleSplit(n_splits=5, test_size=0.3)
for train, test in sss.split(X, y):
    logreg_fit = logreg.fit(X.loc[train],np.ravel(y.loc[train])) 
    y_pred = logreg_fit.predict(X.loc[test])
    print(f1_score(np.ravel(y.loc[test]), y_pred, average=None))

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge



[0.32       1.         0.78125    0.39130435 1.         0.86153846
 0.82191781 0.6        0.96774194 1.        ]
[0.39285714 1.         0.8115942  0.3255814  1.         0.95238095
 0.82191781 0.75471698 0.95238095 1.        ]
[0.375      1.         0.69565217 0.35555556 1.         0.82539683
 0.82191781 0.65517241 0.9375     1.        ]
[0.47058824 1.         0.8        0.27906977 1.         0.95238095
 0.8        0.59259259 0.9375     1.        ]
[0.40816327 1.         0.80597015 0.33333333 1.         0.92063492
 0.85714286 0.74074074 0.86956522 1.        ]


In [177]:
results = [[0.32,1.,0.78125,0.39130435, 1.  ,0.86153846,0.82191781, 0.6   ,     0.96774194, 1.        ],
[0.39285714,1.,0.8115942 , 0.3255814 , 1.   ,0.95238095,0.82191781, 0.75471698, 0.95238095 ,1.        ],
[0.375,1.,0.69565217, 0.35555556 ,1.   ,0.82539683,0.82191781 ,0.65517241, 0.9375   ,  1.        ],
[0.47058824,1.,0.8  ,      0.27906977 ,1.  ,0.95238095,0.8   ,     0.59259259, 0.9375    , 1.        ],
[0.40816327,1.,0.80597015 ,0.33333333, 1.   ,0.92063492,0.85714286 ,0.74074074, 0.86956522 ,1.        ]]

In [178]:
print(lines_to_be_outed)
print(np.mean(results, axis=0))

[1, 4, 7, 10, 14, 18, 21, 24, 30, 34]
[0.39332173 1.         0.7788933  0.33696888 1.         0.90246642
 0.82457926 0.66864454 0.93293762 1.        ]


In [180]:
train_df.to_csv('training_10_lines_100_samples_0325.csv')